XOR

In [1]:
import tensorflow as tf
import numpy as np

tf.set_random_seed(777)  # for reproducibility
learning_rate = 0.1

x_data = [[0, 0],
          [0, 1],
          [1, 0],
          [1, 1]]
y_data = [[0],
          [1],
          [1],
          [0]]
x_data = np.array(x_data, dtype=np.float32)
y_data = np.array(y_data, dtype=np.float32)

X = tf.placeholder(tf.float32, [None, 2])
Y = tf.placeholder(tf.float32, [None, 1])

In [2]:
# Neural Network
def _xor_nn(num_layers = 2, num_deep = 2, iteration=6001):
    W = []
    b = []
    _x = []
    
    _x.append(X)
    for i in range(num_deep - 1):
        _x_dims = tf.cast(_x[i].get_shape().dims[-1], tf.int32)
        W.append( tf.Variable(tf.random_normal([_x_dims, num_layers]), name='weight%d' % (i)) )
        b.append( tf.Variable(tf.random_normal([num_layers]), name='bias%d' % (i)) )
        _x.append( tf.sigmoid(tf.matmul(_x[i], W[i]) + b[i]) )
        print("[%2d]" % i, _x[i].get_shape().dims[-1],
              "X:", _x[i].shape, "W:", W[i].shape, "b:", b[i].shape)
    
    i += 1
    _x_dims = tf.cast(_x[i].get_shape().dims[-1], tf.int32)
    W.append( tf.Variable(tf.random_normal([_x_dims, 1]), name='weight%d' % (i)) )
    b.append( tf.Variable(tf.random_normal([1]), name='bias%d' % (i)) )
    print("[%2d]" % i, _x[i].get_shape().dims[-1],
          "X:", _x[i].shape, "W:", W[i].shape, "b:", b[i].shape)
    hypothesis = tf.sigmoid(tf.matmul(_x[i], W[i]) + b[i])

    # cost/loss function
    cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) *
                       tf.log(1 - hypothesis))

    train = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

    # Accuracy computation
    # True if hypothesis>0.5 else False
    predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))
    
    # Launch graph
    with tf.Session() as sess:
        # Initialize TensorFlow variables
        sess.run(tf.global_variables_initializer())

        for step in range(iteration):
            sess.run(train, feed_dict={X: x_data, Y: y_data})
            if step % 1000 == 0:
                print(step, sess.run(cost, feed_dict={
                      X: x_data, Y: y_data})) # , sess.run([W1, W2])

        # Accuracy report
        h, c, a = sess.run([hypothesis, predicted, accuracy],
                           feed_dict={X: x_data, Y: y_data})
        print("\nHypothesis: ", h, "\nCorrect: ", c, "\nAccuracy: ", a)

In [3]:
_xor_nn()

[ 0] 2 X: (?, 2) W: (2, 2) b: (2,)
[ 1] 2 X: (?, 2) W: (2, 1) b: (1,)
0 0.69486797
1000 0.6928445
2000 0.69087136
3000 0.6784605
4000 0.51510537
5000 0.18575802
6000 0.08090529

Hypothesis:  [[0.05948782]
 [0.9161737 ]
 [0.9161814 ]
 [0.08350647]] 
Correct:  [[0.]
 [1.]
 [1.]
 [0.]] 
Accuracy:  1.0


In [4]:
# wide NN
_xor_nn(4, 2, 4001)

[ 0] 2 X: (?, 2) W: (2, 4) b: (4,)
[ 1] 4 X: (?, 4) W: (4, 1) b: (1,)
0 1.170323
1000 0.5016449
2000 0.17550837
3000 0.07534009
4000 0.0446926

Hypothesis:  [[0.04403021]
 [0.9610225 ]
 [0.9549619 ]
 [0.04677103]] 
Correct:  [[0.]
 [1.]
 [1.]
 [0.]] 
Accuracy:  1.0


In [5]:
# deep NN
_xor_nn(10, 4, 4001)

[ 0] 2 X: (?, 2) W: (2, 10) b: (10,)
[ 1] 10 X: (?, 10) W: (10, 10) b: (10,)
[ 2] 10 X: (?, 10) W: (10, 10) b: (10,)
[ 3] 10 X: (?, 10) W: (10, 1) b: (1,)
0 1.9574454
1000 0.32381663
2000 0.02902981
3000 0.012235245
4000 0.007487577

Hypothesis:  [[0.00824687]
 [0.99249375]
 [0.9928899 ]
 [0.00697467]] 
Correct:  [[0.]
 [1.]
 [1.]
 [0.]] 
Accuracy:  1.0
